In [40]:
import numpy as np

In [41]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10 
NUM_DIMENSIONS = 2 

In [42]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [43]:
CONSTRAINTS

array([[205,  29],
       [195, 141],
       [297, 223]])

In [44]:
import numpy as np
from icecream import ic
from tqdm import trange 

def compute_value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

def is_feasible(total_weights):
    return not np.any(total_weights > CONSTRAINTS)

def random_solution():
    sol = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    total_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))
    for i in np.random.permutation(NUM_ITEMS):
        k = np.random.randint(NUM_KNAPSACKS)
        if np.all(total_weights[k] + WEIGHTS[i] <= CONSTRAINTS[k]):
            sol[k, i] = True
            total_weights[k] += WEIGHTS[i]
    value = compute_value(sol)
    return sol, total_weights, value

def simulated_annealing(max_iter=20000, T0=1000.0, alpha=0.999):
    sol, total_weights, current_val = random_solution()
    best_sol = sol.copy()
    best_val = current_val
    T = T0

    for it in trange(max_iter, desc="Simulated Annealing"):
        # pick random item and random knapsack (-1 = remove)
        i = np.random.randint(NUM_ITEMS)
        old_k_arr = np.where(sol[:, i])[0]
        old_k = old_k_arr[0] if len(old_k_arr) > 0 else None
        k = np.random.randint(-1, NUM_KNAPSACKS)
        if k == old_k: # skip if no actual change
            continue

        new_weights = total_weights.copy()
        if old_k is not None:
            new_weights[old_k] -= WEIGHTS[i]
        if k != -1:
            new_weights[k] += WEIGHTS[i]

        if np.any(new_weights > CONSTRAINTS):
            continue  # infeasible

        # delta cost
        delta = VALUES[i] * ((k != -1) - (old_k is not None))
        new_val = current_val + delta

        if delta > 0 or np.random.random() < np.exp(delta / T):
            # apply move
            if old_k is not None:
                sol[old_k, i] = False
            if k != -1:
                sol[k, i] = True
            total_weights = new_weights
            current_val = new_val

            if new_val > best_val:
                best_val = new_val
                best_sol = sol.copy()

        T *= alpha
        if T < 1e-3:
            break

    return best_sol, best_val

## TEST PROBLEMS

In [45]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing()
ic(best_sol)
ic(best_val)

Simulated Annealing: 100%|██████████| 20000/20000 [00:00<00:00, 57615.97it/s]
ic| best_sol: array([[ True,  True, False, False,  True, False, False,  True,  True,
                      False, False,  True, False, False,  True, False, False,  True,
                       True, False],
                     [False, False,  True,  True, False, False, False, False, False,
                       True,  True, False,  True,  True, False, False, False, False,
                      False, False],
                     [False, False, False, False, False,  True,  True, False, False,
                      False, False, False, False, False, False,  True,  True, False,
                      False,  True]])
ic| best_val: 1065


1065

In [46]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing()
ic(best_sol)
ic(best_val)

Simulated Annealing: 100%|██████████| 20000/20000 [00:00<00:00, 55594.78it/s]
ic| best_sol: array([[False, False, False, False, False, False, False, False, False,
                      False, False, False, False, False, False, False, False, False,
                      False, False, False, False, False, False,  True, False, False,
                       True, False, False, False, False, False, False, False, False,
                      False, False, False, False, False, False, False, False, False,
                      False, False, False, False, False, False,  True, False, False,
                      False, False, False, False, False, False, False, False, False,
                      False, False, False, False, False, False, False, False, False,
                      False, False, False, False,  True, False, False, False, False,
                      False, False, False, False, False, False, False,  True, False,
                      False,  True, False, False, False, False, False, F

48765

In [47]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = simulated_annealing()
ic(best_sol)
ic(best_val)

Simulated Annealing: 100%|██████████| 20000/20000 [00:00<00:00, 39829.96it/s]
ic| best_sol: array([[False, False, False, ..., False, False, False],
                     [False, False, False, ..., False, False, False],
                     [False, False, False, ..., False, False, False],
                     ...,
                     [False, False, False, ..., False, False, False],
                     [False, False, False, ..., False, False, False],
                     [False, False, False, ..., False, False, False]])
ic| best_val: 1196690


1196690